<a href="https://colab.research.google.com/github/peisuke/generative_ai_notebooks/blob/main/12_05_TRL_DPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# パッケージのインストール
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.5.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00


In [3]:
# HuggingFaceのログイン
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM
from datasets import Dataset, load_dataset
from typing import Dict
import torch
from transformers import TrainingArguments
from trl import DPOTrainer
from peft import LoraConfig

In [5]:
# プロンプト+応答からのプロンプトを抽出。
def extract_anthropic_prompt(sample):
    text0 = sample["chosen"]
    text0 = text0.replace("\\n\\n人間:", "User: ")
    text0 = text0.replace("\\n\\nAssistant:", "\n\nAssistant: ")
    text1 = sample["rejected"]
    text1 = text1.replace("\\n\\n人間:", "User: ")
    text1 = text1.replace("\\n\\nAssistant:", "\n\nAssistant: ")
    search_term = "\n\nAssistant: "
    search_term_idx0 = text0.rfind(search_term)
    search_term_idx1 = text1.rfind(search_term)

    return {
        "prompt": text0[: search_term_idx0 + len(search_term)],
        "chosen": text0[search_term_idx0 + len(search_term):],
        "rejected": text1[search_term_idx1 + len(search_term):],
    }
    return sample

In [6]:
dataset = load_dataset("shi3z/anthropic_hh_rlhf_japanese", split="train")
dataset = dataset.map(extract_anthropic_prompt)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/160800 [00:00<?, ? examples/s]

In [7]:
print("--prompt--\n", dataset[0]["prompt"])
print("--chosen--\n", dataset[0]["chosen"])
print("--rejected--\n", dataset[0]["rejected"])

--prompt--
 User: 英語の悪口にはどんなものがありますか?

Assistant: 不完全なリストがあります。\n\nナッシュ、ディック、バガー、ガラクタ、ファック、ガラクタ、雌犬、、頭、袋、陰嚢、、売春婦、ファッカー、食らい、cum、cumbucket、fucknugget、尻穴、うんち、jackass、コックサッカー、、畜生、小便、精子、ブロー、ワンク、ジズム、cum-sucking、自慰、fagot、queer、jzz、jzz-licking、娼婦、slut、ズルチャー、formicator、floozy、wetback、メキシコ人、ヒスパニック、ソドマイト、小人、ママの息子、fagot、変態、クィア、クズ、ビッチ、雌犬、User: 好きなものは何ですか?

Assistant: 
--chosen--
 考えたこともない。
--rejected--
 ロバ。


In [8]:
# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/AI/RLHF/merged_model",  # モデル名
    torch_dtype=torch.float16,
    load_in_4bit=True,
    device_map={"": 0}  # モデル全体をGPU0にロード
)
model.config.use_cache = False  # キャッシュ (学習時はFalse)
model.config.pretraining_tp = 1  # 事前学習で使用したテンソル並列ランク

# 参照モデルの準備
model_ref = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/AI/RLHF/merged_model",  # モデル名
    torch_dtype=torch.float16,
    load_in_4bit=True,
    device_map={"": 0}  # モデル全体をGPU0にロード
)
model_ref.config.pretraining_tp = 1  # 事前学習で使用したテンソル並列ランク

# トークナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-hf",  # モデル名
    use_fast=False,  # Fastトークナイザーの有効化
    add_eos_token=True,  # データへのEOSの追加を指示
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right" # fp16でのオーバーフロー問題対策

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:
# LoRAパラメータ
peft_config = LoraConfig(
    r=64,  # LoRAアテンションの次元
    lora_alpha=16,  # LoRAスケーリングのAlphaパラメータ
    lora_dropout=0.1,  # LoRA レイヤーのドロップアウト確率
    bias="none",  # LoRAのバイアス種別 ("none","all", "lora_only")
    task_type="CAUSAL_LM",  # タスク種別
    target_modules=["q_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "k_proj", "v_proj"]
)

# 学習パラメータ
training_args = TrainingArguments(
    output_dir="./train_logs",  # 出力フォルダ
    fp16=False,  # fp16学習の有効化
    bf16=True,  # bf16学習の有効化
    max_steps=300,  # 学習ステップ数
    per_device_train_batch_size=4,  # 学習用のGPUあたりのバッチサイズ
    gradient_accumulation_steps=1,  # 勾配を蓄積するための更新ステップの数
    optim="paged_adamw_32bit",  # オプティマイザ
    learning_rate=2e-4,  # 初期学習率
    lr_scheduler_type="cosine",  # 学習率スケジュール
    max_grad_norm=0.3,  # 最大法線勾配 (勾配クリッピング)
    warmup_ratio=0.03,  # 線形ウォームアップのステップ比率 (0から学習率まで)
    weight_decay=0.001,  # bias/LayerNormウェイトを除く全レイヤーに適用するウェイト減衰
    save_steps=50,  # 何ステップ毎にチェックポイントを保存するか
    logging_steps=50,  # 何ステップ毎にログを記録するか
    report_to="tensorboard"  # レポート
)

# DPOトレーナーの準備
dpo_trainer = DPOTrainer(
    model,
    model_ref,
    args=training_args,
    beta=0.1,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

# 学習の実行
dpo_trainer.train()
dpo_trainer.model.save_pretrained("/content/drive/MyDrive/AI/RLHF/dpo_model")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:117: UserWarning: When using DPODataCollatorWithPadding, you should set `max_length` in the DPOTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:124: UserWarning: When using DPODataCollatorWithPadding, you should set `max_prompt_length` in the DPOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:143: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for 

Step,Training Loss
50,0.712500
100,0.676200
150,0.687900
200,0.683700
250,0.629400
300,0.608500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.